In [1]:
import os 
os.chdir("../")

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain.document_loaders'

In [3]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

extracted_data = load_pdf_files("data")
print(len(extracted_data))
extracted_data

NameError: name 'DirectoryLoader' is not defined

In [4]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs
minimal_docs = filter_to_minimal_docs(extracted_data)
minimal_docs

ModuleNotFoundError: No module named 'langchain.schema'

In [5]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")
texts_chunk

NameError: name 'minimal_docs' is not defined

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()
embedding

/var/folders/gq/fngx79cs1p34q80fpp8n4x540000gn/T/ipykernel_42713/3442840130.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/venkat/anaconda3/envs/medibot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
import numpy as np
import torch

print("NumPy:", np.__version__)
print("Torch:", torch.__version__)

# critical test
x = torch.randn(3, 4)
y = x.numpy()
print("Torch → NumPy conversion OK:", y.shape)


NumPy: 1.26.4
Torch: 2.2.2
Torch → NumPy conversion OK: (3, 4)


In [8]:
vector = embedding.embed_query("Hello world")
print( "Vector length:", len(vector))
vector

Vector length: 384


[-0.034477319568395615,
 0.031023208051919937,
 0.0067349751479923725,
 0.0261089950799942,
 -0.03936201333999634,
 -0.16030244529247284,
 0.06692400574684143,
 -0.006441435310989618,
 -0.04745043069124222,
 0.01475885882973671,
 0.07087526470422745,
 0.055527638643980026,
 0.019193362444639206,
 -0.026251330971717834,
 -0.010109556838870049,
 -0.026940438896417618,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.017492979764938354,
 0.007676245644688606,
 0.05435226857662201,
 0.003254440613090992,
 0.031725868582725525,
 -0.08462143689393997,
 -0.029406003654003143,
 0.05159556120634079,
 0.0481240376830101,
 -0.0033148236107081175,
 -0.05827918276190758,
 0.04196929931640625,
 0.022210687398910522,
 0.1281888782978058,
 -0.02233893610537052,
 -0.011656295508146286,
 0.06292832642793655,
 -0.03287631645798683,
 -0.09122603386640549,
 -0.031175334006547928,
 0.05269952863454819,
 0.047034792602062225,
 -0.08420310914516449,
 -0.030056169256567955,
 -0.02074481

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [11]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)
pc

In [12]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=texts_chunk,
#     embedding=embedding,
#     index_name=index_name
# )

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore

# # Embed each chunk and upsert the embeddings into your Pinecone index.
# docsearch = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding
# )

docsearch = PineconeVectorStore(
    index=index,
    embedding=embedding,   # SAME embedding model you used earlier
)

In [16]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='8b41d91f-0677-4f14-9cd3-aeb4f76391b3', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='70eb69a8-4db1-42eb-93a4-9ffada4464ac', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='18ef835c-d197-476d-baff-da5e0f3e8b3f', metadata={'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [ ]:
# # from langchain_openai import ChatOpenAI

# # chatModel = ChatOpenAI(model="gpt-4o")

# # from langchain_google_genai import ChatGoogleGenerativeAI

# # # chatModel = ChatGoogleGenerativeAI(
# # #     model="gemini-1.5-flash",
# # #     temperature=0,
# # # )


# # chatModel = ChatGoogleGenerativeAI(
# #     model="gemini-2.0-flash",
# #     temperature=0,
# # )

# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# # chatModel = HuggingFaceEndpoint(
# #     repo_id="tiiuae/falcon-7b-instruct",  # good free model
# #     max_length=512,
# #     temperature=0.3,
# # )

# chatModel = HuggingFaceEndpoint(
#     repo_id='TheBloke/guanaco-7B-HF',
#     task='text-generation',
#     temperature=0.3,
#     max_new_tokens=512,
#     huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
# )
# # chatModel = HuggingFaceEndpoint(
# # repo_id='meta-llama/Llama-2-7b-chat-hf', # Replace as needed
# # task='text-generation',
# # temperature=0.7,
# # max_new_tokens=200
# # )




The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/venkat/.cache/huggingface/token
Login successful


In [61]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_huggingface import HuggingFacePipeline

# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",        # CPU or GPU
#     torch_dtype="auto"
# )

# text_gen_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=256,
#     temperature=0.2,
#     do_sample=True
# )

# chatModel = HuggingFacePipeline(pipeline=text_gen_pipeline)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_core.language_models.llms import LLM
from typing import Optional, List
import torch

class LocalHFLLM(LLM):
    pipeline: any

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        output = self.pipeline(
            prompt,
            max_new_tokens=256,
            temperature=0.2,
            do_sample=True,
        )[0]["generated_text"]
        return output[len(prompt):]

    @property
    def _llm_type(self) -> str:
        return "local_hf"


model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

chatModel = LocalHFLLM(pipeline=hf_pipeline)



/Users/venkat/anaconda3/envs/medibot/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:648: ArbitraryTypeWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warnings.warn(


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
# model = ChatHuggingFace(llm=chatModel)

# result = model.invoke("What is the capital of India?")

# print(result.content)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


HfHubHTTPError: 410 Client Error: Gone for url: https://api-inference.huggingface.co/models/TheBloke/guanaco-7B-HF/v1/chat/completions

https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead.

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# system_prompt = (
#     "You are an Medical assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )
system_prompt = (
    "You are a medical assistant. Use the provided context to answer. "
    "If the answer is not in the context, say you do not know. "
    "Use at most three concise sentences.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


In [46]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

HfHubHTTPError: 410 Client Error: Gone for url: https://api-inference.huggingface.co/models/meta-llama/Llama-3.1-8B-Instruct

https://api-inference.huggingface.co is no longer supported. Please use https://router.huggingface.co instead.

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])